## Ingest circuits.csv file

**Step 1: Read the CSV file using spark dataframe reader**

In [0]:
display(dbutils.fs.mounts())

In [0]:
display(dbutils.fs.ls("/mnt/formulaf1adls/raw"))

In [0]:
circuits_df = spark.read.csv("dbfs:/mnt/formulaf1adls/raw/circuits.csv")

In [0]:
type(circuits_df)

In [0]:
circuits_df.show(n=10,truncate=False)

In [0]:
circuits_df.limit(10).display()

In [0]:
circuits_df = spark.read.option("header",True).csv("dbfs:/mnt/formulaf1adls/raw/circuits.csv")
display(circuits_df)

In [0]:
circuits_df.printSchema()

In [0]:
circuits_df.describe().show()

In [0]:
# infer schema will go throught the data and try to figure out the data type
circuits_df = spark.read \
        .option("header",True) \
        .option("inferSchema",True) \
        .csv("dbfs:/mnt/formulaf1adls/raw/circuits.csv")
circuits_df.printSchema()
# reading the data is 1 spark job and 2nd spark job is going throught the data and infering the datatype which is not efficient

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType 

In [0]:
# StructType is representing rows
# StructFields is representing columns
circuit_schema = StructType(fields=[StructField("circuitId", IntegerType(), False),
                                    StructField("circuitRef", StringType(), True),
                                    StructField("name", StringType(), True),
                                    StructField("location", StringType(), True),
                                    StructField("country", StringType(), True),
                                    StructField("lat", DoubleType(), True),
                                    StructField("lng", DoubleType(), True),
                                    StructField("alt", IntegerType(), True),
                                    StructField("url", StringType(), True)
                                    ])

In [0]:
circuits_df = spark.read \
    .option("header", True) \
        .schema(circuit_schema) \
            .csv("dbfs:/mnt/formulaf1adls/raw/circuits.csv")

In [0]:
circuits_df.limit(10).display()

In [0]:
display(circuits_df.printSchema())
# the standard behaviour of the dataframe reader, if we need to handle nulls,
# that needs to be handled explicitly using other functions

**Step 2: Selecting only the required columns**

In [0]:
circuits_selected_df = circuits_df.select("circuitId", "circuitRef", "name", "location", "country", "lat", "lng", "alt")

In [0]:
circuits_selected_df.limit(10).display()

In [0]:
circuits_selected_df = circuits_df.select(circuits_df.circuitId, circuits_df.circuitRef, circuits_df.name, circuits_df.location, circuits_df.country, circuits_df.lat, circuits_df.lng, circuits_df.alt)
circuits_selected_df.limit(10).display()

In [0]:
from pyspark.sql.functions import col

circuits_selected_df = circuits_df.select(col("circuitId"), col("circuitRef"), col("name"),
                                          col("location"), col("country"), col("lat"), col("lng"), col("alt"))

circuits_selected_df.limit(10).display()

**Step 3: Renaming the columns**

In [0]:
circuits_renanmed_df = circuits_selected_df.withColumnRenamed("circuitId", "circuit_id") \
  .withColumnRenamed("circuitRef", "circuit_ref") \
  .withColumnRenamed("lat", "latitude") \
  .withColumnRenamed("lng", "longitude") \
  .withColumnRenamed("alt", "altitude")



In [0]:
circuits_renanmed_df.limit(5).display()

**Step 4: adding a new column ingestion timestamp**

In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
circuits_final_df = circuits_renanmed_df.withColumn("ingestion_date", current_timestamp())

In [0]:
circuits_final_df.limit(5).display()

**Step 5: Now write the final csv file to the processed container**

In [0]:
circuits_final_df.write.mode("overwrite").parquet("/mnt/formulaf1adls/processed/circuits")

In [0]:
display(spark.read.parquet("/mnt/formulaf1adls/processed/circuits"))